In [1]:
import pandas as pd
import numpy as np

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
import os
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

# Note

from http://www.healthdata.org/gbd/faq

**DALY** is an abbreviation for disability-adjusted life year. It is a universal metric that allows researchers and policymakers to compare very different populations and health conditions across time. DALYs equal the sum of years of life lost (YLLs) and years lived with disability (YLDs). One DALY equals one lost year of healthy life. DALYs allow us to estimate the total number of years lost due to specific causes and risk factors at the country, regional, and global levels.

**Years of life lost (YLLs)** are years lost due to premature mortality. YLLs are calculated by subtracting the age at death from the longest possible life expectancy for a person at that age. For example, if the longest life expectancy for men in a given country is 75, but a man dies of cancer at 65, this would be 10 years of life lost due to cancer.

**YLD** is an abbreviation for years lived with disability, which can also be described as years lived in less than ideal health. This includes conditions such as influenza, which may last for only a few days, or epilepsy, which can last a lifetime. It is measured by taking the prevalence of the condition multiplied by the disability weight for that condition. Disability weights reflect the severity of different conditions and are developed through surveys of the general public.

# Observation

In [3]:
source1 = "./source/IHME-Data-All causes-DALYs.zip"
source2 = "./source/IHME-Data-All risk factors-DALYs.zip"
source3 = "./source/IHME-Data-Global-DALYs.zip"

In [4]:
data1 = pd.read_csv(source1)
data2 = pd.read_csv(source2)
data3 = pd.read_csv(source3)

In [5]:
data3.columns

Index(['﻿location', 'location_name', 'cause', 'cause_name', 'risk',
       'risk_name', 'sex', 'sex_name', 'age', 'age_name', 'year', 'year_name',
       'nm_mean', 'nm_lower', 'nm_upper', 'rt_mean', 'rt_lower', 'rt_upper',
       'pc_mean', 'pc_lower', 'pc_upper', 'metric', 'metric_name'],
      dtype='object')

In [6]:
data1.iloc[:, [1, 3, 5, 7, 9, 11, 12, 15, 18, 21, 22]].head()

,location_name,cause_name,risk_name,sex_name,age_name,year_name,nm_mean,rt_mean,pc_mean,metric,metric_name
0,Global,All causes,NaN,Male,Under 5,1990,5.628233e+08,171218.221456,1.0,2,DALYs
1,Global,All causes,NaN,Female,Under 5,1990,5.035298e+08,161812.044837,1.0,2,DALYs
2,Global,All causes,NaN,Both,Under 5,1990,1.066353e+09,166644.011248,1.0,2,DALYs
3,Global,All causes,NaN,Male,Under 5,1995,4.991255e+08,153827.814903,1.0,2,DALYs
4,Global,All causes,NaN,Female,Under 5,1995,4.424648e+08,145106.962714,1.0,2,DALYs


In [7]:
data2.iloc[:, [1, 3, 5, 7, 9, 11, 12, 15, 18, 21, 22]].head()

,location_name,cause_name,risk_name,sex_name,age_name,year_name,nm_mean,rt_mean,pc_mean,metric,metric_name
0,Global,All causes,All risk factors,Male,Under 5,1990,2.109966e+08,64187.941693,0.374857,2,DALYs
1,Global,All causes,All risk factors,Female,Under 5,1990,2.069637e+08,66508.917679,0.411054,2,DALYs
2,Global,All causes,All risk factors,Both,Under 5,1990,4.179604e+08,65316.628988,0.391952,2,DALYs
3,Global,All causes,All risk factors,Male,Under 5,1995,1.786520e+08,55059.606072,0.357897,2,DALYs
4,Global,All causes,All risk factors,Female,Under 5,1995,1.740846e+08,57091.285607,0.393458,2,DALYs


In [8]:
data3.iloc[:, [1, 3, 5, 7, 9, 11, 12, 15, 18, 21, 22]].head()

,location_name,cause_name,risk_name,sex_name,age_name,year_name,nm_mean,rt_mean,pc_mean,metric,metric_name
0,Global,All causes,NaN,Male,Under 5,1990,5.628233e+08,171218.221456,1.000000,2,DALYs
1,Global,All causes,"Unsafe water, sanitation, and handwashing",Male,Under 5,1990,7.328123e+07,22293.109910,0.130195,2,DALYs
2,Global,All causes,Unsafe water source,Male,Under 5,1990,6.549230e+07,19923.614425,0.116356,2,DALYs
3,Global,All causes,Unsafe sanitation,Male,Under 5,1990,4.754521e+07,14463.874713,0.084470,2,DALYs
4,Global,All causes,Air pollution,Male,Under 5,1990,3.225821e+07,9813.370702,0.057313,2,DALYs


# Compare location/risks/causes files

In [9]:
def cleanup(df, m):
    """return only the columns needed"""
    df.columns = list(map(to_concept_id, df.columns))
    df = df.loc[:, ['location', 'cause', 'risk', 'sex', 'age', 'year',
       'nm_mean', 'nm_lower', 'nm_upper', 'rt_mean', 'rt_lower', 'rt_upper',
       'pc_mean', 'pc_lower', 'pc_upper', 'metric_name']]
    
    assert len(df['metric_name'].unique()) == 1
    assert df['metric_name'].unique()[0] == m
    
    for i in ['location', 'cause', 'risk', 'sex', 'age']:
        df[i] = df[i].astype('category')
        
    return df.drop('metric_name', axis=1).set_index(['location', 'cause', 'risk', 'sex', 'age', 'year'])

In [10]:
# here is a country data file

a = pd.read_csv('source/IHME-Data-Albania-DALYs.zip')
a = cleanup(a, 'DALYs')

In [11]:
a.query('cause == 725 & sex == 3 & age == 27') # cause = Assault by firearm. Note that it contains many risks

nm_mean  nm_lower  nm_upper     rt_mean  \
location cause risk sex age year                                             
43       725   0    3   27  1990  0.001799  0.001221  0.003396  179.861612   
               101  3   27  1990  0.000185  0.000103  0.000372   18.534746   
               102  3   27  1990  0.000185  0.000103  0.000372   18.534746   
               133  3   27  1990  0.000061  0.000024  0.000148    6.069247   
               135  3   27  1990  0.000061  0.000024  0.000148    6.069247   
               169  3   27  1990  0.000240  0.000135  0.000468   23.975944   
               203  3   27  1990  0.000240  0.000135  0.000468   23.975944   
               0    3   27  1995  0.003703  0.002005  0.004515  370.316726   
               101  3   27  1995  0.000501  0.000260  0.000695   50.050798   
               102  3   27  1995  0.000501  0.000260  0.000695   50.050798   
               133  3   27  1995  0.000094  0.000024  0.000195    9.364243   
               135  3   27  1995  0.000094  0.000024  0.000195    9.364243   
               169  3   27  1995  0.000581  0.000335  0.000797   58.122858   
               203  3   27  1995  0.000581  0.000335  0.000797   58.122858   
               0    3   27  2000  0.004034  0.002090  0.005090  403.425752   
               101  3   27  2000  0.000508  0.000252  0.000739   50.813954   
               102  3   27  2000  0.000508  0.000252  0.000739   50.813954   
               133  3   27  2000  0.000163  0.000060  0.000321   16.285877   
               135  3   27  2000  0.000163  0.000060  0.000321   16.285877   
               169  3   27  2000  0.000650  0.000330  0.000934   65.006739   
               203  3   27  2000  0.000650  0.000330  0.000934   65.006739   
               0    3   27  2005  0.003011  0.001955  0.003852  301.112413   
               101  3   27  2005  0.000512  0.000308  0.000700   51.173280   
               102  3   27  2005  0.000512  0.000308  0.000700   51.173280   
               133  3   27  2005  0.000122  0.000036  0.000244   12.198675   
               135  3   27  2005  0.000122  0.000036  0.000244   12.198675   
               169  3   27  2005  0.000612  0.000386  0.000808   61.219437   
               203  3   27  2005  0.000612  0.000386  0.000808   61.219437   
               0    3   27  2010  0.002881  0.001714  0.003959  288.144053   
               101  3   27  2010  0.000535  0.000294  0.000777   53.511010   
               102  3   27  2010  0.000535  0.000294  0.000777   53.511010   
               133  3   27  2010  0.000106  0.000030  0.000227   10.600141   
               135  3   27  2010  0.000106  0.000030  0.000227   10.600141   
               169  3   27  2010  0.000621  0.000357  0.000887   62.134179   
               203  3   27  2010  0.000621  0.000357  0.000887   62.134179   
               0    3   27  2013  0.002780  0.001555  0.003951  277.976937   
               101  3   27  2013  0.000491  0.000270  0.000726   49.147823   
               102  3   27  2013  0.000491  0.000270  0.000726   49.147823   
               133  3   27  2013  0.000128  0.000052  0.000231   12.756932   
               135  3   27  2013  0.000128  0.000052  0.000231   12.756932   
               169  3   27  2013  0.000596  0.000341  0.000855   59.637401   
               203  3   27  2013  0.000596  0.000341  0.000855   59.637401   

                                    rt_lower    rt_upper   pc_mean  pc_lower  \
location cause risk sex age year                                               
43       725   0    3   27  1990  122.059510  339.604639  0.005715  0.003745   
               101  3   27  1990   10.297070   37.178177  0.102779  0.075683   
               102  3   27  1990   10.297070   37.178177  0.102779  0.075683   
               133  3   27  1990    2.404901   14.792807  0.033426  0.016125   
               135  3   27  1990    2.404901   14.792807  0.033426  0.016125   
               169  3   27  1990   13.5054

In [13]:
a.query('cause == 725 & sex == 3 & age == 27').reset_index()['risk'].unique()

array([  0, 101, 102, 133, 135, 169, 203])

In [14]:
# here is a cause file

b = pd.read_csv('source/IHME-Data-Assault by firearm-DALYs.zip') 
b = cleanup(b, 'DALYs')

In [15]:
b.query('location == 43 & sex == 3 & age == 27')  # location = Albania. Note that it only contains risk = 0

nm_mean  nm_lower  nm_upper     rt_mean  \
location cause risk sex age year                                             
43       725   0    3   27  1990  0.001799  0.001221  0.003396  179.861612   
                            1995  0.003703  0.002005  0.004515  370.316726   
                            2000  0.004034  0.002090  0.005090  403.425752   
                            2005  0.003011  0.001955  0.003852  301.112413   
                            2010  0.002881  0.001714  0.003959  288.144053   
                            2013  0.002780  0.001555  0.003951  277.976937   

                                    rt_lower    rt_upper   pc_mean  pc_lower  \
location cause risk sex age year                                               
43       725   0    3   27  1990  122.059510  339.604639  0.005715  0.003745   
                            1995  200.514959  451.503818  0.011977  0.006460   
                            2000  208.967383  508.964931  0.013515  0.006892   
                            2005  195.517244  385.230888  0.010374  0.006635   
                            2010  171.424266  395.921086  0.010440  0.006333   
                            2013  155.475200  395.087542  0.010414  0.006140   

                                  pc_upper  
location cause risk sex age year            
43       725   0    3   27  1990  0.010641  
                            1995  0.015185  
                            2000  0.017749  
                            2005  0.013330  
                            2010  0.013844  
                            2013  0.014086

In [16]:
# we can get same result from the location file.

a.query('cause == 725 & sex == 3 & age == 27 & risk == 0')

nm_mean  nm_lower  nm_upper     rt_mean  \
location cause risk sex age year                                             
43       725   0    3   27  1990  0.001799  0.001221  0.003396  179.861612   
                            1995  0.003703  0.002005  0.004515  370.316726   
                            2000  0.004034  0.002090  0.005090  403.425752   
                            2005  0.003011  0.001955  0.003852  301.112413   
                            2010  0.002881  0.001714  0.003959  288.144053   
                            2013  0.002780  0.001555  0.003951  277.976937   

                                    rt_lower    rt_upper   pc_mean  pc_lower  \
location cause risk sex age year                                               
43       725   0    3   27  1990  122.059510  339.604639  0.005715  0.003745   
                            1995  200.514959  451.503818  0.011977  0.006460   
                            2000  208.967383  508.964931  0.013515  0.006892   
                            2005  195.517244  385.230888  0.010374  0.006635   
                            2010  171.424266  395.921086  0.010440  0.006333   
                            2013  155.475200  395.087542  0.010414  0.006140   

                                  pc_upper  
location cause risk sex age year            
43       725   0    3   27  1990  0.010641  
                            1995  0.015185  
                            2000  0.017749  
                            2005  0.013330  
                            2010  0.013844  
                            2013  0.014086

In [18]:
# here is a risk file.

c = pd.read_csv('source/IHME-Data-Alcohol use-DALYs.zip')
c = cleanup(c, 'DALYs')

In [20]:
c.query('location == 43 & sex == 3 & age == 27')  # only contains cause = all causes

nm_mean  nm_lower  nm_upper      rt_mean  \
location cause risk sex age year                                              
43       294   102  3   27  1990  0.008004  0.006476  0.009698   800.395739   
                            1995  0.009171  0.007390  0.010934   917.119037   
                            2000  0.008670  0.007004  0.010338   866.993651   
                            2005  0.010662  0.008557  0.012671  1066.246996   
                            2010  0.010564  0.008252  0.013041  1056.434135   
                            2013  0.009481  0.007248  0.012007   948.120942   

                                    rt_lower     rt_upper   pc_mean  pc_lower  \
location cause risk sex age year                                                
43       294   102  3   27  1990  647.602551   969.780704  0.025395  0.020589   
                            1995  738.989501  1093.380906  0.029622  0.023915   
                            2000  700.423607  1033.807885  0.029020  0.023731   
                            2005  855.722563  1267.116438  0.036686  0.029920   
                            2010  825.231943  1304.077418  0.038290  0.031066   
                            2013  724.849401  1200.706359  0.035594  0.028395   

                                  pc_upper  
location cause risk sex age year            
43       294   102  3   27  1990  0.030776  
                            1995  0.035496  
                            2000  0.034724  
                            2005  0.043271  
                            2010  0.046020  
                            2013  0.043034

In [23]:
a.query('cause == 294 & sex == 3 & age == 27 & risk == 102') # also can be select from location file.

nm_mean  nm_lower  nm_upper      rt_mean  \
location cause risk sex age year                                              
43       294   102  3   27  1990  0.008004  0.006476  0.009698   800.395739   
                            1995  0.009171  0.007390  0.010934   917.119037   
                            2000  0.008670  0.007004  0.010338   866.993651   
                            2005  0.010662  0.008557  0.012671  1066.246996   
                            2010  0.010564  0.008252  0.013041  1056.434135   
                            2013  0.009481  0.007248  0.012007   948.120942   

                                    rt_lower     rt_upper   pc_mean  pc_lower  \
location cause risk sex age year                                                
43       294   102  3   27  1990  647.602551   969.780704  0.025395  0.020589   
                            1995  738.989501  1093.380906  0.029622  0.023915   
                            2000  700.423607  1033.807885  0.029020  0.023731   
                            2005  855.722563  1267.116438  0.036686  0.029920   
                            2010  825.231943  1304.077418  0.038290  0.031066   
                            2013  724.849401  1200.706359  0.035594  0.028395   

                                  pc_upper  
location cause risk sex age year            
43       294   102  3   27  1990  0.030776  
                            1995  0.035496  
                            2000  0.034724  
                            2005  0.043271  
                            2010  0.046020  
                            2013  0.043034

## conclution

cause/risk file contains cause/risk data for all country and all risk/cause. only country data are complete.

# Codebook

In [67]:
cb = pd.read_csv('source/codebook/IHME_GBD_2013_CODEBOOK_Y2016M01D15.CSV', encoding='latin')

In [68]:
cb = cb.drop([0])

In [68]:
cb.columns

Index(['Variable:', 'location', 'location_name', 'cause', 'cause_name', 'risk',
       'risk_name', 'sex', 'sex_name', 'age', 'age_name', 'year', 'year_name',
       'nm_mean', 'nm_lower', 'nm_upper', 'rt_mean', 'rt_lower', 'rt_upper',
       'pc_mean', 'pc_lower', 'pc_upper', 'metric', 'metric_name'],
      dtype='object')

In [123]:
fs = [x for x in os.listdir('source/') if 'DALYs' in x]
fs2 = fs.copy()

In [100]:
from unidecode import unidecode
import re

In [69]:
loc = cb['location_name'].dropna()
loc[loc.duplicated()]

Series([], Name: location_name, dtype: object)

In [124]:
for c in cb['location_name'].dropna().values:
    found = 0
    fn = 'IHME-Data-{}-.*.zip'.format(unidecode(c))
    for i in fs:
        if re.match(fn, i):
            found = 1
            try:
                fs2.remove(i)
            except:
                print(i)
            
    if found == 0:
        print(c)

IHME-Data-North Africa and Middle East-DALYs.zip
IHME-Data-South Asia-DALYs.zip
Cote d'Ivoire
IHME-Data-Guinea-Bissau-DALYs.zip
East Asia & Pacific
Europe & Central Asia
IHME-Data-Latin America and Caribbean-DALYs.zip
IHME-Data-Sub-Saharan Africa-DALYs.zip
Middle East & North Africa
IHME-Data-South Asia-DALYs.zip
WB region
IHME-Data-Mexico-DALYs.zip
Michoacán de Ocampo
Nuevo León
Querétaro
San Luis Potosí
Yucatán
World Bank Lower Income


In [67]:
# above 3 are in the files.

# Cote d'Ivoire => Cote dIvoire
# WB region => World bank region
# World Bank Lower Income => World Bank Low Income

In [115]:
cn = cb['cause_name'].dropna()
cn[cn.duplicated()]

Series([], Name: cause_name, dtype: object)

In [125]:
for c in cb['cause_name'].dropna().values:
    found = 0
    fn = 'IHME-Data-{}-.*.zip'.format(unidecode(c))
    for i in fs:
        if re.match(fn, i):
#             print(i)
            found = 1
            fs2.remove(i)
            
    if found == 0:
        print(c)

HIV/AIDS and tuberculosis
HIV/AIDS
HIV/AIDS resulting in mycobacterial infection
HIV/AIDS resulting in other diseases
Tracheal, bronchus and lung cancer
Attention-deficit/hyperactivity disorder
Age-related and other hearing loss
Exposure to forces of nature
Maternal deaths aggravated by HIV/AIDS


In [75]:
# above is missing

In [116]:
cn = cb['risk_name'].dropna()
cn[cn.duplicated()]

Series([], Name: risk_name, dtype: object)

In [126]:
for c in cb['risk_name'].dropna().values:
    found = 0
    fn = 'IHME-Data-{}-.*.zip'.format(unidecode(c))
    for i in fs:
        if re.match(fn, i):
            found = 1
            try:
                fs2.remove(i)
            except:
                print(i)
            
    if found == 0:
        print(c)

IHME-Data-Vitamin A deficiency-DALYs.zip
Metabolic
Occupational exposure to second-hand smoke
Environmental risks


In [127]:
# Vitamin A deficiency: duplicated in cause_name

In [77]:
# Occupational exposure to second-hand smoke => Occupational exposure to secondhand smoke

In [78]:
len(cb['location_name'].dropna())

343

In [79]:
len(cb['cause_name'].dropna())

307

In [80]:
len(cb['risk_name'].dropna())

79

In [81]:
343+307+79

729

In [128]:
len(fs2)

27

In [129]:
fs2

['IHME-Data-Attention-deficit+hyperactivity disorder-DALYs.zip',
 'IHME-Data-Cote dIvoire-DALYs.zip',
 'IHME-Data-East Asia & Pacific - WB-DALYs.zip',
 'IHME-Data-Environmental+occupational risks-DALYs.zip',
 'IHME-Data-Europe & Central Asia - WB-DALYs.zip',
 'IHME-Data-Exposure to forces of nature, disaster-DALYs.zip',
 'IHME-Data-HIV+AIDS and tuberculosis-DALYs.zip',
 'IHME-Data-HIV+AIDS resulting in mycobacterial infection-DALYs.zip',
 'IHME-Data-HIV+AIDS resulting in other diseases-DALYs.zip',
 'IHME-Data-HIV+AIDS-DALYs.zip',
 'IHME-Data-Latin America & Caribbean - WB-DALYs.zip',
 'IHME-Data-Maternal deaths aggravated by HIV+AIDS-DALYs.zip',
 'IHME-Data-Metabolic risks-DALYs.zip',
 'IHME-Data-México-DALYs.zip',
 'IHME-Data-Michoacán de Ocampo-DALYs.zip',
 'IHME-Data-Middle East & North Africa - WB-DALYs.zip',
 'IHME-Data-Nuevo León-DALYs.zip',
 'IHME-Data-Occupational exposure to secondhand smoke-DALYs.zip',
 'IHME-Data-Other hearing loss-DALYs.zip',
 'IHME-Data-Querétaro-DALYs

In [37]:
d = {  # TODO
    "Cote d'Ivoire": 'IHME-Data-Cote dIvoire-YLDs.zip',
    "East Asia & Pacific": 'IHME-Data-East Asia & Pacific - WB-YLDs.zip',
    "Europe & Central Asia": 'IHME-Data-Europe & Central Asia - WB-YLDs.zip',
    "Middle East & North Africa":  'IHME-Data-Middle East & North Africa - WB-YLDs.zip',
    "WB region":  'IHME-Data-World Bank Regions-YLDs.zip',
    "Michoacán de Ocampo":  'IHME-Data-Michoacán de Ocampo-YLDs.zip',
    "Nuevo León":  'IHME-Data-Nuevo León-YLDs.zip',
    "Querétaro":  'IHME-Data-Querétaro-YLDs.zip',
    "San Luis Potosí":  'IHME-Data-San Luis Potosí-YLDs.zip',
    "Yucatán":  'IHME-Data-Yucatán-YLDs.zip',
    "World Bank Lower Income":  'IHME-Data-World Bank Low Income-YLDs.zip'
}

* IHME-Data-North Africa and Middle East-DALYs.zip
* IHME-Data-South Asia-DALYs.zip
* Cote d'Ivoire
* IHME-Data-Guinea-Bissau-DALYs.zip
* East Asia & Pacific
* Europe & Central Asia
* IHME-Data-Latin America and Caribbean-DALYs.zip
* IHME-Data-Sub-Saharan Africa-DALYs.zip
* Middle East & North Africa
* IHME-Data-South Asia-DALYs.zip
* WB region
* IHME-Data-Mexico-DALYs.zip
* Michoacán de Ocampo
* Nuevo León
* Querétaro
* San Luis Potosí
* Yucatán
* World Bank Lower Income
* IHME-Data-Vitamin A deficiency-DALYs.zip
* Metabolic
* Occupational exposure to second-hand smoke
* Environmental risks
* HIV/AIDS and tuberculosis
* HIV/AIDS
* HIV/AIDS resulting in mycobacterial infection
* HIV/AIDS resulting in other diseases
* Tracheal, bronchus and lung cancer
* Attention-deficit/hyperactivity disorder
* Age-related and other hearing loss
* Exposure to forces of nature
* Maternal deaths aggravated by HIV/AIDS

In [ ]:
"Cote d'Ivoire"
"East Asia & Pacific"
"Europe & Central Asia"
"Middle East & North Africa"
"WB region"
"Michoacán de Ocampo"
"Nuevo León"
"Querétaro"
"San Luis Potosí"
"Yucatán"
"World Bank Lower Income"

In [65]:
dup1 = pd.read_csv('source/IHME-Data-Mali-DALYs.zip')

In [66]:
dup1.iloc[:, 0].unique()

array([211])